# Similarities among reference plasmids in test samples

Here, we examine the similarities of the reference plasmids (occurring in the test samples) with each other, in order to check whether we try to predict the same plasmids or very similar ones in different samples over and over again. 
This could bias the evaluation scores in one way or the other, depending on whether we would be able to consistenly predict such potentially frequent plasmids or not.

This analysis only uses the reference plasmids and is independent of the gene databases used in the evaluation.

*Summary:*   
25 reference plasmids formed 6 clusters of identical sequences. 
17 of these plasmids and 15 others also participated in near-identical matches.

In [2]:
from Bio import SeqIO, Seq

import numpy as np
import pandas as pd
import subprocess

pd.options.display.max_rows = None

In [3]:
sample_script = '/project/6007976/wg-anoph/Plasmids-Assembly/data/2018-05-23__MOB-suite_benchmark_reads/sample.sh'
extract_script = '/project/6007976/wg-anoph/Plasmids-Assembly/data/2018-05-17__MOB-suite_benchmark/extract.sh'

load_modules = 'module load gcc/5.4.0 blast+/2.6.0;'
unload_modules = 'module unload blast+/2.6.0 gcc/5.4.0;'


test_ids = [1,5,15,16,18,19,23,24,25,26,27,28,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,50,51,52,55,56,62,63,64,65,66,76,85,86,87,102,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,129,133]
ref_file = 'references.fasta'
map_file = 'mapping.csv'
blast_db = 'plasmid_references'
length_threshold = 0.95
identity_threshold = 0.95

subprocess.call('rm -f %s' % ref_file, shell = True)
cnt = 0
for sample_id in test_ids:
    p = subprocess.Popen('bash %s pla %s' % (sample_script, sample_id), stdout = subprocess.PIPE, shell = True)
    output, _ = p.communicate()
    p.wait()
    plasmids = output.rstrip().decode().split(',')
    cnt += len(plasmids)
    
    for acc in plasmids:
        file = 'tmp.fasta'
        subprocess.call('bash %s %s %s; cat %s >> %s; rm %s' % (extract_script, acc, file, file, ref_file, file), shell = True)

subprocess.call(load_modules + 'makeblastdb -in %s -dbtype nucl -out %s; ' % (ref_file, blast_db) + unload_modules, shell = True)
    
print('%s should contain %i sequences.' % (ref_file, cnt))

references = dict()
with open(ref_file, 'r') as in_file:
    for record in SeqIO.parse(in_file, 'fasta'):
        references[record.id] = record.seq
        
print('%i sequences read from %s' % (len(references), ref_file))

subprocess.call(load_modules + 'blastn -task megablast -query %s -db %s -out %s -outfmt 6; ' % (ref_file, blast_db, map_file) + unload_modules, shell = True)

matches = pd.read_csv(map_file, sep = '\t', header = None)
matches.columns = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']

references.fasta should contain 147 sequences.
147 sequences read from references.fasta


## Identical sequences with different identifiers

Checks the reverse complement as well.

In [4]:
contents = []
for i in references:
    contents.append([i, str(references[i])])
df_references = pd.DataFrame(contents, columns = ['id', 'seq'])

refs_with_identical_match = set()
seq_groups = df_references.groupby(['seq'])
for s, grp in seq_groups:
    rc = Seq.reverse_complement(s)
    rc_grp = seq_groups.get_group(rc) if rc in seq_groups.groups else []
    if (len(grp) + len(rc_grp)) > 1:
        print('Forward:\n', grp['id'])
        for acc in grp['id']:
            refs_with_identical_match.add(acc)
        if len(rc_grp):
            print('Reverse complement:\n', rc_grp['id'])
            for acc in rc_grp['id']:
                refs_with_identical_match.add(acc)
        print('Sequence is %i nt long.\n' % len(s))

Forward:
 88    CP016575.1
Name: id, dtype: object
Reverse complement:
 131    CP012926.1
Name: id, dtype: object
Sequence is 37697 nt long.

Forward:
 33    CP016570.1
69    CP016577.1
71    CP016566.1
76    CP016574.1
Name: id, dtype: object
Sequence is 2246 nt long.

Forward:
 131    CP012926.1
Name: id, dtype: object
Reverse complement:
 88    CP016575.1
Name: id, dtype: object
Sequence is 37697 nt long.

Forward:
 3      CP016515.1
6      CP016578.1
22     CP012922.1
42     CP016506.1
50     CP016567.1
54     CP016531.1
70     CP016524.1
98     CP016564.1
102    CP016562.1
107    CP016571.1
140    CP016509.1
Name: id, dtype: object
Sequence is 37697 nt long.

Forward:
 32     CP016580.1
138    CP016587.1
Name: id, dtype: object
Sequence is 37696 nt long.

Forward:
 86     CP012934.1
115    CP012927.1
Name: id, dtype: object
Sequence is 3372 nt long.

Forward:
 14    CP016508.1
18    CP016505.1
62    CP012925.1
64    CP016523.1
Name: id, dtype: object
Sequence is 2096 nt long.



First and third group are the same. Thus, there are 6 different clusters of identical reference plasmids, involving 25 plasmids.

## Near-identical sequences

Two sequences are near-identical, when they match each other with an identity of at least 95 % (but not 100 %) and when the match covers at least 95 % of both sequences.

In [5]:
filtered_matches = matches.copy()

# remove self matches
filtered_matches = filtered_matches[filtered_matches['qseqid'] != filtered_matches['sseqid']] 

# remove matches that are too short or are too dissimilar
filtered_matches = filtered_matches.loc[filtered_matches['pident'] >= (identity_threshold * 100)]
align_lengths = np.array(filtered_matches['length'].tolist())
query_lengths = np.array([len(references[q]) for q in filtered_matches['qseqid']])
db_lengths = np.array([len(references[q]) for q in filtered_matches['sseqid']])
too_short = (align_lengths / query_lengths < length_threshold) | (align_lengths / db_lengths < length_threshold)
filtered_out_length = filtered_matches.loc[too_short]
filtered_matches = filtered_matches.loc[~too_short]

# keep the matches only in one direction
filtered_matches = filtered_matches[filtered_matches['qseqid'] < filtered_matches['sseqid']]

filtered_matches.drop(['evalue', 'bitscore'], axis = 1, inplace = True)
filtered_matches['qseq_len'] = [len(references[i]) for i in filtered_matches['qseqid']]
filtered_matches['sseq_len'] = [len(references[i]) for i in filtered_matches['sseqid']]

In [6]:
non_identical_filtered_matches = filtered_matches.copy()
non_identical_filtered_matches = non_identical_filtered_matches[
    (non_identical_filtered_matches['pident'] < 100)
    | (non_identical_filtered_matches['length'] != non_identical_filtered_matches['qseq_len'])
    | (non_identical_filtered_matches['length'] != non_identical_filtered_matches['sseq_len'])
    | (non_identical_filtered_matches['mismatch'] != 0)
    | (non_identical_filtered_matches['gapopen'] != 0)]

In [8]:
non_identical_filtered_matches

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
16342,AP017618.1,AP017621.1,99.997,116530,2,2,1,116529,1,116529,116529,116529
81056,CP017437.1,CP017447.1,99.994,92565,0,1,1,92565,1,92559,92565,92559
81083,CP017437.1,CP017443.1,99.981,92583,0,1,1,92565,1,92583,92565,92583
81110,CP017437.1,CP017439.1,99.967,92596,0,2,1,92565,1,92596,92565,92596
82400,CP017439.1,CP017443.1,99.986,92596,0,2,1,92596,1,92583,92596,92583
83745,CP017441.1,CP017445.1,99.983,92735,1,2,1,92729,1,92726,92729,92726
85077,CP017443.1,CP017447.1,99.974,92583,0,1,1,92583,1,92559,92583,92559
129239,CP016515.1,CP016518.1,99.997,37698,0,1,1,37697,1,37698,37697,37698
129248,CP016515.1,CP016580.1,99.997,37697,0,1,1,37697,1,37696,37697,37696
129257,CP016515.1,CP016587.1,99.997,37697,0,1,1,37697,1,37696,37697,37696


*Number of 'participating' plasmids in the near-identical matches*

In [9]:
len(set(non_identical_filtered_matches['qseqid']) | set(non_identical_filtered_matches['sseqid']))

32

*Overlap with the plasmids already identified in the identical matches*

In [11]:
len((set(non_identical_filtered_matches['qseqid']) | set(non_identical_filtered_matches['sseqid'])) & refs_with_identical_match)

17

*Number of 'participating' plasmids in identical and near-identical matches*

In [12]:
len(set(non_identical_filtered_matches['qseqid']) | set(non_identical_filtered_matches['sseqid']) | refs_with_identical_match)

40

### Near-identical matches per qseqid

**AP017618.1**

In [13]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'AP017618.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
16342,AP017618.1,AP017621.1,99.997,116530,2,2,1,116529,1,116529,116529,116529


**CP017437.1**

In [14]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP017437.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
81056,CP017437.1,CP017447.1,99.994,92565,0,1,1,92565,1,92559,92565,92559
81083,CP017437.1,CP017443.1,99.981,92583,0,1,1,92565,1,92583,92565,92583
81110,CP017437.1,CP017439.1,99.967,92596,0,2,1,92565,1,92596,92565,92596


**CP017439.1**

In [15]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP017439.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
82400,CP017439.1,CP017443.1,99.986,92596,0,2,1,92596,1,92583,92596,92583


**CP017441.1**

In [16]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP017441.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
83745,CP017441.1,CP017445.1,99.983,92735,1,2,1,92729,1,92726,92729,92726


**CP017443.1**

In [17]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP017443.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
85077,CP017443.1,CP017447.1,99.974,92583,0,1,1,92583,1,92559,92583,92559


**CP016515.1**

In [18]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP016515.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
129239,CP016515.1,CP016518.1,99.997,37698,0,1,1,37697,1,37698,37697,37698
129248,CP016515.1,CP016580.1,99.997,37697,0,1,1,37697,1,37696,37697,37696
129257,CP016515.1,CP016587.1,99.997,37697,0,1,1,37697,1,37696,37697,37696


CP016580.1 and CP016587.1 are identical.

**CP016516.1**

In [19]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP016516.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
129386,CP016516.1,CP016568.1,99.994,98998,6,0,1,98998,1,98998,98998,98998


**CP012925.1**

In [20]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP012925.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
130084,CP012925.1,CP012932.1,99.809,2096,4,0,1,2096,1,2096,2096,2096


**CP016562.1**

In [21]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP016562.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
132261,CP016562.1,CP016580.1,99.997,37697,0,1,1,37697,1,37696,37697,37696
132270,CP016562.1,CP016587.1,99.997,37697,0,1,1,37697,1,37696,37697,37696


CP016580.1 and CP016587.1 are identical.

**CP016564.1**

In [22]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP016564.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
132498,CP016564.1,CP016580.1,99.997,37697,0,1,1,37697,1,37696,37697,37696
132507,CP016564.1,CP016587.1,99.997,37697,0,1,1,37697,1,37696,37697,37696


CP016580.1 and CP016587.1 are identical.

**CP016567.1**

In [23]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP016567.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
132814,CP016567.1,CP016580.1,99.997,37697,0,1,1,37697,1,37696,37697,37696
132823,CP016567.1,CP016587.1,99.997,37697,0,1,1,37697,1,37696,37697,37696


CP016580.1 and CP016587.1 are identical.

**CP016571.1**

In [24]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP016571.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
133833,CP016571.1,CP016580.1,99.997,37697,0,1,1,37697,1,37696,37697,37696
133842,CP016571.1,CP016587.1,99.997,37697,0,1,1,37697,1,37696,37697,37696


CP016580.1 and CP016587.1 are identical.

**CP016578.1**

In [25]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP016578.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
135184,CP016578.1,CP016580.1,99.997,37697,0,1,1,37697,1,37696,37697,37696
135193,CP016578.1,CP016587.1,99.997,37697,0,1,1,37697,1,37696,37697,37696


CP016580.1 and CP016587.1 are identical.

**CP016518.1**

In [26]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP016518.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
135349,CP016518.1,CP016524.1,99.997,37698,0,1,1,37698,1,37697,37698,37697
135358,CP016518.1,CP016531.1,99.997,37698,0,1,1,37698,1,37697,37698,37697
135367,CP016518.1,CP016578.1,99.997,37698,0,1,1,37698,1,37697,37698,37697
135376,CP016518.1,CP016571.1,99.997,37698,0,1,1,37698,1,37697,37698,37697
135385,CP016518.1,CP016567.1,99.997,37698,0,1,1,37698,1,37697,37698,37697
135394,CP016518.1,CP016564.1,99.997,37698,0,1,1,37698,1,37697,37698,37697
135403,CP016518.1,CP016562.1,99.997,37698,0,1,1,37698,1,37697,37698,37697
135421,CP016518.1,CP016580.1,99.995,37698,0,2,1,37698,1,37696,37698,37696
135430,CP016518.1,CP016587.1,99.995,37698,0,2,1,37698,1,37696,37698,37696


CP016580.1 and CP016587.1 are identical.

**CP012932.1**

In [27]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP012932.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
137671,CP012932.1,CP016505.1,99.809,2096,4,0,1,2096,1,2096,2096,2096
137672,CP012932.1,CP016508.1,99.809,2096,4,0,1,2096,1,2096,2096,2096
137673,CP012932.1,CP016523.1,99.809,2096,4,0,1,2096,1,2096,2096,2096


CP016505.1, CP016508.1 and CP016523.1 are identical.

**CP012936.1**

In [28]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP012936.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
137745,CP012936.1,CP016572.1,99.944,99000,53,2,1,98999,1,98999,98999,98999


**CP016531.1**

In [29]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP016531.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
138549,CP016531.1,CP016580.1,99.997,37697,0,1,1,37697,1,37696,37697,37696
138558,CP016531.1,CP016587.1,99.997,37697,0,1,1,37697,1,37696,37697,37696


CP016580.1 and CP016587.1 are identical.

**CP016524.1**

In [30]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP016524.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
140376,CP016524.1,CP016580.1,99.997,37697,0,1,1,37697,1,37696,37697,37696
140385,CP016524.1,CP016587.1,99.997,37697,0,1,1,37697,1,37696,37697,37696


CP016580.1 and CP016587.1 are identical.

**CP012922.1**

In [31]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP012922.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
140604,CP012922.1,CP016518.1,99.997,37698,0,1,1,37697,1,37698,37697,37698
140613,CP012922.1,CP016580.1,99.997,37697,0,1,1,37697,1,37696,37697,37696
140622,CP012922.1,CP016587.1,99.997,37697,0,1,1,37697,1,37696,37697,37696


CP016580.1 and CP016587.1 are identical.

**CP012923.1**

In [32]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP012923.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
140751,CP012923.1,CP016516.1,99.938,99011,48,2,1,99011,1,98998,99011,98998
140760,CP012923.1,CP016568.1,99.932,99011,54,2,1,99011,1,98998,99011,98998


**CP016509.1**

In [33]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP016509.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
141580,CP016509.1,CP016518.1,99.997,37698,0,1,1,37697,1,37698,37697,37698
141589,CP016509.1,CP016580.1,99.997,37697,0,1,1,37697,1,37696,37697,37696
141598,CP016509.1,CP016587.1,99.997,37697,0,1,1,37697,1,37696,37697,37696


CP016580.1 and CP016587.1 are identical.

**CP016506.1**

In [34]:
non_identical_filtered_matches[non_identical_filtered_matches['qseqid'] == 'CP016506.1']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,qseq_len,sseq_len
141852,CP016506.1,CP016518.1,99.997,37698,0,1,1,37697,1,37698,37697,37698
141861,CP016506.1,CP016580.1,99.997,37697,0,1,1,37697,1,37696,37697,37696
141870,CP016506.1,CP016587.1,99.997,37697,0,1,1,37697,1,37696,37697,37696


CP016580.1 and CP016587.1 are identical.

### Larger group of sequences of length 37697 +/- 1

In [35]:
ref_ids = set(filtered_matches['qseqid']) | set(filtered_matches['sseqid'])
seqs = set()
seq_groups = dict()
for i in ref_ids:
    s = str(references[i])
    rc = str(Seq.reverse_complement(s))
    if 37696 <= len(s) <= 37698:
        seqs.add(s)
        if s in seq_groups:
            seq_groups[s].append(i)
        elif rc in seq_groups:
            seq_groups[rc].append(i)
        else:
            seq_groups[s] = [i]   
            
n = 1
for s in seq_groups:
    print('Group %i, sequence length %i, represented by %s:\n' % (n, len(s), seq_groups[s][0]), seq_groups[s], '\n')
    n += 1

Group 1, sequence length 37696, represented by CP016580.1:
 ['CP016580.1', 'CP016587.1'] 

Group 2, sequence length 37697, represented by CP016562.1:
 ['CP016562.1', 'CP016515.1', 'CP016578.1', 'CP016509.1', 'CP016564.1', 'CP016571.1', 'CP012922.1', 'CP016506.1', 'CP016567.1', 'CP016531.1', 'CP016524.1'] 

Group 3, sequence length 37697, represented by CP016575.1:
 ['CP016575.1', 'CP012926.1'] 

Group 4, sequence length 37698, represented by CP016518.1:
 ['CP016518.1'] 



*Group 4 vs Group 1:*
17702 nt equal, A-deletion, 5856 nt equal, A-deletion, 14138 nt equal
    
*Group 4 vs Group 2:*
23559 nt equal, A-deletion, 14138 nt equal
    
*Group 1 vs Group 2:*
17702 nt equal, A-insertion, 19994 nt equal

*Group 3:*
The two sequences are reverse complements of each other.

*Group 2 vs Group 3:*
CP012926.1[13666:37697] + CP012926.1[1:13665] = CP016562.1

--> Group 1 + A at position 17703 = Group 2; Group 2 + A at position 23560 = Group 4   
--> Group 2 is circular shifted version of Group 3
